In [7]:
from src.data._dataobject import DataObject
from src.data.types._text import Text
from src.data.types._nested import Nested
from src.data.types._list import List, Array

class Address(DataObject):
    city = Text()
    street = Text()

class City(DataObject):
    name = Text(is_id=True, nullable=True)


class User(DataObject):
    __table__ = 'users'

    first_name = Text()
    last_name = Text()
    address = Nested(Address)
    cities = List(City)

u = User(
    first_name='Frodo',
    last_name='Baggins',
    address=Address(
        city='Piacenza',
        street='Via San Sisto'
    ),
    cities=[City(name='Milano'), City(name='Piacenza'), City(name='Kigali')]
)


In [8]:
City(name='')
City.name

<Text name=None>

In [9]:
a = Array(int, [5])

In [10]:
u.cities[0].name

'Milano'

In [11]:
a.append(3)

In [12]:
a

[5, 3]

In [13]:
u.to_dict()

{'address': {'city': 'Piacenza', 'street': 'Via San Sisto'},
 'cities': [{'name': 'Milano'}, {'name': 'Piacenza'}, {'name': 'Kigali'}],
 'first_name': 'Frodo',
 'last_name': 'Baggins'}

In [14]:
c = City(name='Piacenza')

In [15]:
c

<City name=Piacenza>

In [16]:
City.__annotations__

{'name': str}

In [17]:
City()

TypeError: <lambda>() missing 1 required positional argument: 'name'